## Fastai 3D images


### Resources

* https://towardsdatascience.com/working-with-3d-data-fastai2-5e2baa09037e 
* https://www.kaggle.com/jhoward/some-dicom-gotchas-to-be-aware-of-fastai
* https://forums.fast.ai/t/fastai-v2-has-a-medical-imaging-submodule/56117
* https://towardsdatascience.com/deep-learning-with-magnetic-resonance-and-computed-tomography-images-e9f32273dcb5

Note that Fastai defaults to DICOM and doesn't appear to support NIFTI. 

In [2]:
from fastai.data.all import *
from fastai.vision.all import *

In [17]:
%load_ext autoreload
%autoreload 2

from utils import *



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Write a DataBlock. 

It will give you back a Dataset and DataLoader


To build a DataBlock you need to give the library four things: 
 
* the types of your input/labels, and at least two functions: `get_items` and `splitter`. 
* You may also need to include `get_x` and `get_y` or a more generic list of getters that are applied to the results of get_items.



In [ ]:
#example DataBlock - won't run.

dblock = DataBlock(
    blocks    = (ImageSequenceBlock, CategoryBlock),
    get_items = SequenceGetItems('file', 'sequence_id', 'label'), 
    get_x     = lambda t : t[:-1],
    get_y     = lambda t : t[-1],
    splitter  = RandomSplitter())

In [ ]:
#this is the definition of an ImageBlock in the fast.ai codebase.
def ImageBlock(cls=PILImage):
    "A `TransformBlock` for images of `cls`"
    return TransformBlock(type_tfms=cls.create, batch_tfms=IntToFloatTensor)


In [ ]:
#what is the format of get_items? 

In [8]:
#https://docs.fast.ai/data.transforms.html#get_files
files = get_files('./data/small/t1', extensions='.gz', recurse=False)

In [9]:
files[0]

Path('data/small/t1/IXI167-HH-1569-T1_fcm.nii.gz')

In [11]:
type(files)

fastcore.foundation.L

From the first link in the resource section:

> remember from the DataBlock example above that get_x and get_y receive the output from get_items and should separate what is the input and what is the target. In this case, they are as simple as this:

In [20]:


class NiftiGetItems():

    def __init__(self):
        pass
        
    def __call__(self, source_dir):
        
        #TODO: could just take the parent directory
        t1_files = os.listdir(source_dir)

        subj_list = [ path_to_subj(e) for e in t1_files if fnmatch(e, r'*T1_fcm.nii.gz') ] 
        
        out = [] 
        
        for s in subj_list:
            t1 = load_mr_image(s, "t1")
            t2 = load_mr_image(s, "t2")           
            out.append({'source': t1, 'target': t2})

        return out


#https://fastcore.fast.ai/transform#Transform
class NiftiToTensor(Transform):
    pass



#This class is only needed if you decide to make GetItems just return paths. 
def NiftiBlock():
    return TransformBlock(item_tfms=NiftiToTensor(),
                          #batch_tfms= crop or data augment here...
                          #type_tfms= path to nifti obj. 
                         )
    

In [21]:
dblock = DataBlock(
    #blocks    = NiftiBlock(),
    get_items = NiftiGetItems(),
    get_x     = lambda t : t['source'], 
    get_y     = lambda t : t['target'],
    #n_inp = how many inputs do you want to pass into the datablock / get_items, etc. 
    splitter  = RandomSplitter())

#TODO: See above. 
# - crop is not happening. Fix this. 
# - not creating tensors. 

In [22]:
dsets = dblock.datasets('./data/small/t1')

In [23]:
dsets

(#28) [(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2cd850>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2cd520>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2cddf0>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2b2a30>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2fb460>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2b2820>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2b2eb0>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2cdfd0>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2da0a0>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2da580>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2da3d0>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2da130>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2dae20>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2de1c0>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2de040>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2de250>),(<nibabel.nifti1.Nifti1Image object at 0x7ffa7d2de490>, <nibabel.nifti1.Nifti1Image object at 0x7ffa7d2de

In [24]:
type(dsets)

fastai.data.core.Datasets

In [25]:
dsets.train[0]

(<nibabel.nifti1.Nifti1Image at 0x7ffa7d2da280>,
 <nibabel.nifti1.Nifti1Image at 0x7ffa7d2f0100>)